In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Feb  6 12:49:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 15977293594165255507474753607285.jpg
 15977302833956435656154636193474.jpg
 15983287715081185679987536013333.jpg
 15983288187695984735239785603316.jpg
 15983308525677145430028985179545.jpg
 15983329016004035836081507659779.jpg
 15985922712113949242991840814882.jpg
 1598592326949214481128133778834.jpg
 20200818_103228.jpg
 +2.jpg
 assignment-1.docx
 assignment1.py
 assignment2.py
 azure.gdoc
 backend-cv.gdoc
 BBCNews.gsheet
'Class 8 Evaluation Final.gsheet'
 Classroom
 clean_data.gsheet
 cleaned_tweets.gsheet
 CNN.gdoc
'Colab Notebooks'
'concept paper (1).gdoc'
'concept paper.gdoc'
 cover_letter_2.pdf
 cover_letter.gdoc
'cover letter.internship.gdoc'
 cover_letter.pdf
 CV_assessment
 CV-ML.pdf
'Data Entry'
 dataentry.gsheet
 Data_jan25.zip
 devraj_resume.pdf
 Drinks.gsheet
 emergency_vs_non-emergency_dataset.zip
 EnglishTweets.gsheet
'Getting started.pdf'
'Group Assignment II - The Business Plan and Strategic Management.gdoc'
 Gyanas-Luitel_OBQ.pdf
'Image Aug

CLONE THE DARKNET

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (10353/10353), done.


COMPILE DARKNET USING NVIDIA GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

CONFIGURE DARKNET NETWORK FOR TRAINING YOLO V3

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "human" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-02-06 13:20:11--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  56.6MB/s    in 2.7s    

2022-02-06 13:20:14 (56.6 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



EXTRACT IMAGES

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/2008_000008.jpg  
  inflating: data/obj/2008_000008.txt  
  inflating: data/obj/2008_000036.jpg  
  inflating: data/obj/2008_000036.txt  
  inflating: data/obj/2008_000041.jpg  
  inflating: data/obj/2008_000041.txt  
  inflating: data/obj/2008_000096.jpg  
  inflating: data/obj/2008_000096.txt  
  inflating: data/obj/2008_000109.jpg  
  inflating: data/obj/2008_000109.txt  
  inflating: data/obj/2008_000128.jpg  
  inflating: data/obj/2008_000128.txt  
  inflating: data/obj/2008_000132.jpg  
  inflating: data/obj/2008_000132.txt  
  inflating: data/obj/2008_000142.jpg  
  inflating: data/obj/2008_000142.txt  
  inflating: data/obj/2008_000143.jpg  
  inflating: data/obj/2008_000143.txt  
  inflating: data/obj/2008_000144.jpg  
  inflating: data/obj/2008_000144.txt  
  inflating: data/obj/2008_000176.jpg  
  inflating: data/obj/2008_000176.txt  
  inflating: data/obj/2008_000199.jpg  
  inflating: data/obj/2008_000199.txt  
  i

In [ ]:
import glob
import os, re

In [ ]:
txt_file_paths = glob.glob(r"data/obj/*.txt")
print(len(txt_file_paths))

160


In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1 0.160000 0.702153 0.164000 0.471292

['1', '0.160000', '0.702153', '0.164000', '0.471292']
0 data/obj/2008_000970.txt
0 0.160000 0.702153 0.164000 0.471292
1 0.297000 0.698565 0.186000 0.483254

['1', '0.297000', '0.698565', '0.186000', '0.483254']
0 data/obj/2008_000970.txt
0 0.297000 0.698565 0.186000 0.483254
1 0.410000 0.691388 0.160000 0.454545

['1', '0.410000', '0.691388', '0.160000', '0.454545']
0 data/obj/2008_000970.txt
0 0.410000 0.691388 0.160000 0.454545
1 0.522000 0.698565 0.160000 0.449761

['1', '0.522000', '0.698565', '0.160000', '0.449761']
0 data/obj/2008_000970.txt
0 0.522000 0.698565 0.160000 0.449761
1 0.664000 0.702153 0.172000 0.437799

['1', '0.664000', '0.702153', '0.172000', '0.437799']
0 data/obj/2008_000970.txt
0 0.664000 0.702153 0.172000 0.437799
1 0.804000 0.692584 0.164000 0.433014

['1', '0.804000', '0.692584', '0.164000', '0.433014']
0 data/obj/2008_000970.txt
0 0.804000 0.692584 0.164000 0.433014
1 0.657000 0.501333 0.670000 0.997333

['1', '0.6570

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(len(images_list))

160


In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

START THE TRAINING

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.379588), count: 2, class_loss = 0.505839, iou_loss = 0.899207, total_loss = 1.405045 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000020, iou_loss = 0.000000, total_loss = 0.000020 
 total_bbox = 79162, rewritten_bbox = 0.120007 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.704710), count: 5, class_loss = 1.076746, iou_loss = 0.470879, total_loss = 1.547625 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.697441), count: 2, class_loss = 0.468681, iou_loss = 0.124658, total_loss = 0.593338 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000020, iou_loss = 0.000000, total_loss = 0.000020 
 total_bbox = 79169, rewritten_bbox = 0.119996 % 
v3 (mse